In [1]:
import glob
import os
import numpy as np

In [2]:
#-----------------------------------------------------------------------------
# Initialize the problem data
#-----------------------------------------------------------------------------

# Read the input data file.

def read_data(filename) :
    eva_tree, graph = [],[]
    nb_node = 0
    mode,start = 0,0
    with open(filename, 'r') as fp :
        for line in fp :
          
            if (line.find('evacuation info') != -1):
#             print('Add evacuation info')             
                mode = 1
                start = 0
            
            elif (line.find('graph') != -1):
#             print('Add graph info')             
                mode = 2
                start = 0
            
            elif (mode == 1 and start == 0) :
                start = 1
                nb_evac_node = int(line.split()[0]) 
                safe_node = int(line.split()[1])
#             print('safe node is : ',safe_node)
#             print('number of evacuation nodes : ', nb_evac_node)
                          
            elif (mode == 2 and start == 0) :
                start = 1
                nb_node = int(line.split()[0])
                n = int(line.split()[1])
#             print('number of nodes : ',nb_node)
#             print('number of edges : ',n)

            
            elif (mode == 1 and start == 1) :
                data = list(map(lambda x : int(x),line.split()))
                eva_tree.append(data)

                  
            elif (mode == 2 and start == 1) :
                data = list(map(lambda x : int(x),line.split()))
                graph.append(data)
      
            
    return eva_tree,graph,nb_node


filename = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.data"
EVA_TREE,GRAPH,NB_NODES = read_data(filename)

In [3]:
def read_solution (filename):
    solution = []
    with open(filename,'r') as fp :
        line = fp.read().splitlines()
        test_name = line[0]
        nb_evac_node = int(line[1])
        for i in range(nb_evac_node) : 
            data = list(map(lambda x : int(x),line[2+i].split()))
            solution.append(data)
            next = 2+i
        nature = line[next+1]
        f_objectif = int(line[next+2])
        methode = line[next+3]
    return solution,f_objectif,nature

filename = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.solution"
SOLUTION,F_OBJECTIF,NATURE = read_solution(filename)

In [4]:
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

print('evacuation_tree = ', EVA_TREE)
print('graph = ',GRAPH)
print('number of evacuation nodes : ',NB_EVA_NODES)
print('List of evac nodes : ',LIST_EVA_NODES)
print('number of nodes = ',NB_NODES)
print('number of edges = ',NB_EDGES)
print('-----------------------------')
print('A solution is :',SOLUTION)
print('f_objectif = ',F_OBJECTIF)
print('Nature of solution : ',NATURE)

evacuation_tree =  [[2, 30, 5, 3, 4, 5, 6], [1, 48, 8, 3, 4, 5, 6], [3, 33, 3, 2, 5, 6]]
graph =  [[1, 4, 13, 7, 8], [2, 4, 26, 4, 5], [3, 5, 28, 6, 3], [4, 5, 33, 9, 10], [5, 6, 46, 12, 11]]
number of evacuation nodes :  3
List of evac nodes :  [2, 1, 3]
number of nodes =  6
number of edges =  5
-----------------------------
A solution is : [[1, 8, 3], [2, 5, 0], [3, 3, 0]]
f_objectif =  37
Nature of solution :  valid


In [5]:
def get_eva_node_info(node_id) :
    [eva_node_info] = [item for item in EVA_TREE if item[0]==node_id]
    nb_evacuees = eva_node_info[1]
    max_rate = eva_node_info[2]
    route_length = eva_node_info[3]
    route_list = eva_node_info[4:]
    return nb_evacuees,max_rate,route_length,route_list

get_eva_node_info(3)

(33, 3, 2, [5, 6])

In [12]:
def get_edge_info(node1,node2):
    [edge_info] = [item for item in GRAPH if (item[0]==node1) & (item[1]==node2)]
    due_date = edge_info[2]
    length = edge_info[3]
    capacity = edge_info[4]
    return due_date,length,capacity

get_edge_info(1,4)

(13, 7, 8)

In [7]:
def get_solution_info(node_id) : 
    [sol_info] = [item for item in SOLUTION if item[:][0] == node_id]
    eva_rate = sol_info[1]
    t_start = sol_info[2]
    return eva_rate,t_start

rate,start = get_solution_info(2)
print(rate,start)

5 0


In [8]:
def get_duration(node_id) :
    nb_evacuees,cap_max,route_len,route_list = get_eva_node_info(node_id)
    
    E_tmp =  nb_evacuees / cap_max
    node_current = node_id
    for i in route_list:
        node_next = i
#         [edge_info] = [item for item in GRAPH if (item[0]==node_current) & (item[1]==node_next)]
        _,length,edge_cap= get_edge_info(node_current,node_next)
        assert(cap_max <= edge_cap)
        E_tmp += length
        # reload the next edge
        node_current = node_next
    #end for
    return int(E_tmp)

get_duration(1)

34

In [67]:
def get_task(node_id,eva_rate=None) :
    tasks = []
    nb_evacuees,max_rate,route_len,route_list = get_eva_node_info(node_id)
    if eva_rate == None : 
        eva_rate = max_rate
    else :
        if (eva_rate > max_rate) :
            print("ERROR ON EVACUATION RATE !!")
#         assert(eva_rate <= max_rate)
    duration = int(nb_evacuees/eva_rate)
    demande_res = [node_id] + route_list
#     print(DEMANDE_RES)
    return duration,demande_res

print(get_task(1,8)[0])
[item for item in get_task(1,8)[1]]

6


[1, 4, 5, 6]

In [52]:
for i in range(NB_EVA_NODES) : 
    node_id = EVA_TREE[i][0]
    print('Evacuation of (only) node ',node_id,' is ', get_duration(node_id),' mins')
    print('Task generated by this node : ',get_task(node_id))

Evacuation of (only) node  2  is  31  mins
Task generated by this node :  (6, [2, 4, 5, 6])
Evacuation of (only) node  1  is  34  mins
Task generated by this node :  (6, [1, 4, 5, 6])
Evacuation of (only) node  3  is  29  mins
Task generated by this node :  (11, [3, 5, 6])


In [73]:
def verify_solution() : 
    
    print('evacuation_tree = ', EVA_TREE)
    print('graph = ',GRAPH)
    print('number of evacuation nodes : ',NB_EVA_NODES)
    print('List of evac nodes : ',LIST_EVA_NODES)
    print('number of nodes = ',NB_NODES)
    print('number of edges = ',NB_EDGES)
    print('-----------------------------')
    print('A solution is :',SOLUTION)
    print('f_objectif = ',F_OBJECTIF)
    print('Nature of solution : ',NATURE)
    print('-----------------------------')
    print('CREATION TASKS & VERIFY CONSTRAINTS')
    print('-----------------------------')
    ressources = {}
    for edge in GRAPH :
        edge_cap = edge[-1]
#         print('max cap of edge [{}-{}] : {}'.format(edge[0],edge[1],edge_cap))
        ressources.setdefault('Cap of edge[{}-{}]'.format(edge[0],edge[1]),np.full(40,edge_cap))
#         print(ressources)
    
    tasks = {}
    for i in LIST_EVA_NODES : 
        nb_evacuees,max_rate,route_length,route_list = get_eva_node_info(i)
        rate,start = get_solution_info(i)
        if rate > max_rate : 
            result = False
            exit(0)
#         print('task {},rate={},start={}'.format(i,rate,start))
        duration,demande_res = get_task(i,rate)
#         print(duration,demande_res)
        current = i
        for j in demande_res : 
#             print('Evacuees from {} at node {}'.format(i,j))
            nxt = j
            if current != nxt :
                _,length,edge_cap = get_edge_info(current,nxt)
                tasks.setdefault('Evacuees from {} at edge [{}-{}]'.format(i,current,nxt), [start,start+length+duration,duration,rate])
                dispo = ressources['Cap of edge[{}-{}]'.format(current,nxt)]
                dispo[start:start+duration] -= rate
#                 print('dispo[{}-{}]='.format(current,nxt),dispo)
                if(dispo.any() < 0) :    
                    print('TOO MANY PERSONS AT EDGE [{}-{}] !!!'.format(current,nxt))
                    result = False
                    exit(0)
#                 assert(dispo.all() >= 0)
                ressources['Cap of edge[{}-{}]'.format(current,nxt)] = dispo
                start += length
            current = nxt
            
    print('ressources info after evacuation = ',ressources)        
    print('tasks = ', tasks)
    print('Nb of tasks = ',len(tasks))
    print('-----------------------------')
    print('CHECK F_OBJECTIF')
    print('-----------------------------')
    

    end_time = np.max([tasks[keys][1] for keys in tasks])
    print('End time is : ',end_time)
    if (end_time == F_OBJECTIF) : 
        result = True
    else :
        result = False
    
    return result

verify_solution()

evacuation_tree =  [[2, 30, 5, 3, 4, 5, 6], [1, 48, 8, 3, 4, 5, 6], [3, 33, 3, 2, 5, 6]]
graph =  [[1, 4, 13, 7, 8], [2, 4, 26, 4, 5], [3, 5, 28, 6, 3], [4, 5, 33, 9, 10], [5, 6, 46, 12, 11]]
number of evacuation nodes :  3
List of evac nodes :  [2, 1, 3]
number of nodes =  6
number of edges =  5
-----------------------------
A solution is : [[1, 8, 3], [2, 5, 0], [3, 3, 0]]
f_objectif =  37
Nature of solution :  valid
-----------------------------
CREATION TASKS & VERIFY CONSTRAINTS
-----------------------------
ressources info after evacuation =  {'Cap of edge[1-4]': array([8, 8, 8, 0, 0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]), 'Cap of edge[2-4]': array([0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]), 'Cap of edge[3-5]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

True